In [1]:
!ls data

Applications.csv       Candidates.csv	    Psychometrics.csv  Vacants.csv
ApplicationStages.csv  LinkRetoTecnico.txt  Stages.csv


In [2]:
from pyspark import SparkConf, SparkContext, SQLContext, HiveContext
from pyspark.sql import SparkSession
import pyspark
import os

import statistics
import json

In [3]:
conf = SparkConf() \
    .setAppName("hackaton2") \
    .setMaster('spark://localhost:7077') \
    .set("spark.executor.memory", "2g") \
    .set("spark.driver.memory", "2g") \
    .set("spark.memory.offHeap.enabled","true") \
    .set("spark.memory.offHeap.size","1g") \
    .set("spark.sql.warehouse.dir", "hdfs://localhost:9000/user/hive/warehouse")

#.set("hive.metastore.uris", "thrift://127.0.0.1:9083") \

In [4]:
!rm metastore_db -rf || rm derby.log

In [5]:
sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
hiveContext = HiveContext(sc)

In [6]:
spark = SparkSession.builder \
    .enableHiveSupport() \
    .getOrCreate()

In [7]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [8]:
hiveContext.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



# Configuraciones

In [9]:
mode = "overwrite"

# Importamos candidates

In [10]:
candidates = hiveContext.read \
    .option("wholeFile", "true") \
    .option("multiLine", "true") \
    .option("escape", "\"") \
    .csv(
        "file:///home/juliotorres/hackaton/data/Candidates.csv",
        sep=',', quote='"'
    )

In [12]:
candidates.show(5)

+---+--------------------+---------------+------+----------+----------+----+---+----------+--------+-------------+--------------------+----+----+-----+-----+----+----+----+----+----+----+----+
|_c0|                 _c1|            _c2|   _c3|       _c4|       _c5| _c6|_c7|       _c8|     _c9|         _c10|                _c11|_c12|_c13| _c14| _c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|
+---+--------------------+---------------+------+----------+----------+----+---+----------+--------+-------------+--------------------+----+----+-----+-----+----+----+----+----+----+----+----+
|
|
|
|
|
+---+--------------------+---------------+------+----------+----------+----+---+----------+--------+-------------+--------------------+----+----+-----+-----+----+----+----+----+----+----+----+
only showing top 5 rows



In [14]:
candidates.write.parquet("/user/juliotorres/candidates", mode=mode)

Py4JJavaError: An error occurred while calling o93.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.

In [ ]:
candidates = candidates.selectExpr(
    "cast(_c0 as int) id", 
    "_c1 email",
    "_c2 first_name",
    "_c3 last_name",
    "_c4 phone",
    "TO_DATE(CAST(UNIX_TIMESTAMP(_c5, 'yyyy-MM-dd') AS TIMESTAMP)) as birthdate",
    "_c6 gender",
    "cast(_c7 as int) identification_type",
    "_c8 identification_number",
    "_c9 country_birth",
    "_c10 city",
    "_c11 education_level",
    "cast(_c12 as int) salary",
    "_c13 profile_description",
    "cast(cast(_c14 as boolean) as int) withow_experience",
    "cast(cast(_c15 as boolean) as int) withow_studies",
    "_c16 sectors",
    "cast(cast(_c17 as boolean) as int) title_of_profetion",
    "_c18 civil_status",
    "_c19 presentation",
    "_c20 educational_institution",
    "_c21 experience",
    "regexp_replace(_c22, '\r', '') profession"
).filter("length(_c2) < 100 and length(_c3) < 100")

ocandidates = candidates

In [ ]:
hiveContext.registerDataFrameAsTable(candidates, "candidates")

In [ ]:
salary_m = hiveContext.sql("""
select percentile_approx(salary, 0.5) salary
  from candidates
 where salary is not null
 """).collect()[0].salary

salary_m

In [ ]:
candidates = candidates \
    .selectExpr(
        "id",
    
        """email      <> '' and email is not null as has_email""",
        """first_name <> '' and first_name is not null as has_first_name""",
        """last_name  <> '' and last_name is not null as has_last_name""",
        """phone      <> '' and phone is not null as has_phone""",

        """CAST(datediff(
            current_date(), 
            TO_DATE(CAST(UNIX_TIMESTAMP(birthdate,'yyyy-MM-dd') AS TIMESTAMP))
           )/365 as integer) as age""",

        """case when gender = 'male' then 'm'
                when gender = 'female' then 'f' 
           else 'u' end as gender""",

        """case when identification_type is null then 100 
           else identification_type 
           end identification_type""",

        """identification_number <> '' and 
           identification_number is not null 
           as has_identification_number""",

        """case when city <> '' and city is not null then city 
           else 'unknow' end as city""",

        """case when education_level <> '' and education_level is not null then education_level 
           else 'unknow' end as education_level""",

        """case when salary is null then %s 
           else salary end as salary""" % salary_m,

        """case when profile_description is null then ''
           else profile_description end as profile_description""",

        """case when withow_experience is null then false 
           else withow_experience = 1 end withow_experience""",

        """case when withow_studies is null then false 
           else withow_studies = 1 end withow_studies""",

        """case when sectors is null then 'oficios varios'
           else sectors end as sectors""",

        """case when title_of_profetion is null then false 
           else title_of_profetion = 1 end title_of_profetion""",

        """case when civil_status is null then 'unknow'
           else civil_status end civil_status""",

        """presentation is not null has_presentation""",
        "presentation",
        # jalar metadata

        "educational_institution", "experience", "profession"
    )

In [ ]:
fields = [ 
    'id', 'has_email', 'has_first_name', 'has_last_name', 'has_phone', 'age', 'gender', 'identification_type',
    'has_identification_number', 'city', 'education_level', 'salary', 'profile_description',
    'withow_experience', 'withow_studies', 'sectors', 'title_of_profetion', 'civil_status', 'presentation'
]
#candidates.columns[1:-3]

In [ ]:
fields[:5]

In [ ]:
def parse_study(r):
    educational_institution = []
    try:
        if r.educational_institution is not None:
            educational_institution = json.loads(r.educational_institution)
    except:
        pass
    return {
        'institute_complete': [ 
            x['institute'] or ''
            for x in educational_institution
            if x['in_progress'] == False
        ],
        'institute_incomplete': [ 
            x['institute'] or ''
            for x in educational_institution
            if x['in_progress'] == True
        ],
        'title_complete': [
            x['title'] or ''
            for x in educational_institution
            if x['in_progress'] == False
        ],
        'title_incomplete': [
            x['title'] or ''
            for x in educational_institution
            if x['in_progress'] == True
        ],
        'study_type_complete': [ 
            x['study_type'] or ''
            for x in educational_institution
            if x['in_progress'] == False
        ],
        'study_type_incomplete': [ 
            x['study_type'] or ''
            for x in educational_institution
            if x['in_progress'] == True
        ]
    }

In [ ]:
from datetime import datetime
import re

In [ ]:
replace = re.compile("\s+")

def parse_experience(r):
    experience = []
    try:
        if r.experience is not None:
            experience = json.loads(r.experience)
    except:
        pass
    
    company = []
    position = []
    description = []
    
    total_experience = 0
        
    if len(experience) > 0: 
        for x in experience:
            start_date = datetime.strptime(x['start_date'], '%Y-%m-%d')
            
            if x['at_present'] != True:
                end_date = datetime.strptime(x['end_date'], '%Y-%m-%d')
            else:
                end_date = datetime.now()
                
            if start_date is not None and end_date is not None:
                num_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)
            else:
                num_months = 1
            
            if x['company']:
                company.append((x['company'] + ' ') * num_months)
            if x['position']:
                position.append((x['position'] + ' ') * num_months)
            if x['description']:
                description.append((replace.sub(' ', x['description']) + ' ') * num_months)
                
            total_experience+= num_months
                
            
    return {
        'company':     company,
        'position':    position,
        'description': description,
        'experience':  total_experience or 0
    }


In [ ]:
parse_experience(candidates.where("experience <> '[]'").limit(1).collect()[0])['experience']

In [ ]:
def parse_profession(r):
    profession = []
    try:
        if r.profession is not None:
            profession = json.loads(r.profession)
    except:
        pass
    
    if len(profession) == 0:
        evaluation = [0]
    elif len(profession) < 3:
        evaluation = [
            sum([
                x['percent'] or 0
                for x in element['psy_tests'] 
            ])
            for element in profession
        ]
    else:
        evaluation = [
            sum([
                x['percent'] or 0
                for x in element['psy_tests'] 
                if x['percent'] is not None
            ])
            for element in profession[-3:]
        ]
    
    evaluation = statistics.median(evaluation) or 0
    
    return {
        'city_name':     [ x['city_name'] or '' for x in profession ],
        'position_name': [ x['position_name'] or '' for x in profession ],
        'evaluation':    int(evaluation)
    }


In [ ]:
mediana = candidates \
    .where('age >= 18 or age > 100') \
    .selectExpr("percentile_approx(age, .5) mediana") \
    .collect()[0].mediana
mediana

In [ ]:
def parse(r):
    s = parse_study(r)
    e = parse_experience(r)
    p = parse_profession(r)
    age = r.age or mediana
    return (
        r.id,r.has_email, r.has_first_name, r.has_last_name, r.has_phone, 
        (age if age <= 80 else mediana) if age >= 18 else mediana,
        r.gender, r.identification_type, r.has_identification_number, 
        r.city, r.education_level, r.salary, r.profile_description, 
        r.withow_experience, r.withow_studies, r.sectors, 
        r.title_of_profetion, r.civil_status, r.presentation,
        ' '.join(s['institute_complete']), 
        ' '.join(s['institute_incomplete']),
        ' '.join(s['title_complete']),
        ' '.join(s['title_incomplete']),
        ' '.join(s['study_type_complete']),
        ' '.join(s['study_type_incomplete']),
        ' '.join(e['company']),
        ' '.join(e['position']),
        ' '.join(e['description']),
        #' '.join(e['experience']),
        ' '.join(p['city_name']),
        ' '.join(p['position_name']),
        p['evaluation'],
    )

candidates = candidates \
    .rdd \
    .map(parse) \
    .toDF(fields+[
        'educational_institute_complete', 
        'educational_institute_incomplete',
        'educational_title_complete', 
        'educational_title_incomplete',
        'educational_study_type_complete',
        'educational_study_type_incomplete',

        'experience_company', 
        'experience_position', 
        'experience_description',
        #'experience_total',

        'profesion_city_name',
        'profesion_position_name',
        'profesion_evaluation'
    ])

In [ ]:
candidates.write.parquet("/user/juliotorres/candidates", mode=mode)

In [ ]:
hiveContext.sql("show tables").show()

In [ ]:
candidates.limit(10).toPandas()

```python
candidates.write.jdbc(url=url, table="candidates", mode=mode, properties=properties)
```

## Creando diccionario de palabras

In [ ]:
def word_counts(r):
    return r

word_counts(candidates.where('experience_description').limit(5).collect()[0])

# Importamos vacants

In [ ]:
vacants = hiveContext.read \
    .option("wholeFile", "true") \
    .option("multiLine", "true") \
    .option("escape", "\"") \
    .csv("file:///home/juliotorres/hackaton/data/Vacants.csv")

In [ ]:
relacion = vacants \
    .where('_c5 is not null') \
    .selectExpr("cast(_c5 as int) _c5", "cast(_c4 as int) _c4") \
    .selectExpr("_c5 / _c4 relacion" ) \
    .selectExpr("percentile_approx(relacion, .7) relacion") \
    .collect()[0].relacion

relacion

In [ ]:
vacants = vacants.selectExpr("*", "regexp_replace(_c18, '\r', '') _c19")

In [ ]:
[ x['_c15'] for x in vacants.select("_c15").distinct().collect()[:5] ]

In [ ]:
vacants = vacants.selectExpr(
    "cast(_c0 as int) id", 
    "case when _c1 is not null then _c1 else '' end title",
    "case when _c2 is not null then _c2 else '' end description",
    "case when _c3 is not null then _c3 else '' end salary_type",
    "case when _c4 is not null then cast(_c4 as int) else '' end min_salary",
    """case when _c5 is not null then cast(_c5 as int) 
       else cast(cast(_c4 as int) * %s as int) end as max_salary""" % (relacion,),
    "cast(_c6 as int) status",
    "TO_DATE(CAST(UNIX_TIMESTAMP(_c7, 'yyyy-MM-dd') AS TIMESTAMP)) as created_at",
    "_c8 company",
    "_c9 education_level",
    "cast(cast(_c10 as boolean) as int) agree",
    "case when _c11 is not null then _c11 else '' end requirements",
    "TO_DATE(CAST(UNIX_TIMESTAMP(_c12, 'yyyy-MM-dd') AS TIMESTAMP)) publish_state",
    "cast(cast(_c13 as boolean) as int) confidential",
    "TO_DATE(CAST(UNIX_TIMESTAMP(_c14, 'yyyy-MM-dd') AS TIMESTAMP)) expiration_date",
    "case when _c15 is not null then _c15 else '' end experience_and_positions",
    "case when _c16 is not null then _c16 else '' end knowledge_and_skills",
    "case when _c17 is not null then _c17 else '' end titles_and_studies",
    "case when _c19 <> '' then cast(_c19 as int) else 0 end number_of_quotas",
)

In [ ]:
vacants.toPandas().experience_and_positions.unique()

In [ ]:
fields = vacants.columns
fields[:4]

In [ ]:
import html2text, re

In [ ]:
def parse_description(t):
    return ' '.join(re.split('\s+', (html2text.html2text(t))))

In [ ]:
def parse(r):
    return (
        r.id,r.title,parse_description(r.description),r.salary_type,r.min_salary,r.max_salary,
        r.status,r.created_at,r.company,r.education_level,r.agree, r.requirements,
        r.publish_state,r.confidential,r.expiration_date or '',
        r.experience_and_positions,r.knowledge_and_skills,r.titles_and_studies,
        r.number_of_quotas,
    )

vacants = vacants.rdd.map(parse).toDF(fields)

In [ ]:
vacants.write.parquet("/user/juliotorres/vacants", mode=mode)

```python
vacants.write.jdbc(url=url, table="vacants", mode=mode, properties=properties)
```

# Importamos stages

In [ ]:
stages = hiveContext.read \
    .option("wholeFile", "true") \
    .option("multiLine", "true") \
    .option("escape", "\"") \
    .csv("file:///home/juliotorres/hackaton/data/Stages.csv")

In [ ]:
stages = stages.selectExpr(
    "cast(_c0 as int) id", 
    "case when _c1 is not null then _c1 else '' end title",
    "cast(cast(_c2 as boolean) as int) send_sms",
    "cast(cast(_c3 as boolean) as int) send_email",
    "cast(cast(_c4 as boolean) as int) send_call",
    "cast(_c5 as int) created_at",
    "cast(_c6 as int) status",
    "cast(0+_c7 as int) discard_type",
)

In [ ]:
stages.write.parquet("/user/juliotorres/stages", mode=mode)

```python
stages.write.jdbc(url=url, table="stages", mode=mode, properties=properties)
```

# Importamos applications

In [ ]:
applications = hiveContext.read \
    .option("wholeFile", "true") \
    .option("multiLine", "true") \
    .option("escape", "\"") \
    .csv("file:///home/juliotorres/hackaton/data/Applications.csv")

In [ ]:
applications = applications.selectExpr(
    "cast(_c0 as int) id", 
    "cast(_c1 as int) as vacant_id",
    "cast(_c2 as int) candidate_id",
    "TO_DATE(CAST(UNIX_TIMESTAMP(_c3, 'yyyy-MM-dd') AS TIMESTAMP)) as created_at",
    "_c4 status",
    "regexp_replace(_c5, '\r', '') register_type"
)

In [ ]:
applications.show(5)

In [ ]:
applications.write.parquet("/user/juliotorres/applications", mode=mode)

```python
applications.write.jdbc(url=url, table="applications", mode=mode, properties=properties)
```

# Importamos applicationStages

In [ ]:
applicationStages = hiveContext.read \
    .option("wholeFile", "true") \
    .option("multiLine", "true") \
    .option("escape", "\"") \
    .csv("file:///home/juliotorres/hackaton/data/ApplicationStages.csv")

In [ ]:
applicationStages = applicationStages.selectExpr(
    "cast(_c0 as int) id", 
    "cast(_c1 as int) stage_id",
    "cast(_c2 as int) application_id",
    "TO_DATE(CAST(UNIX_TIMESTAMP(_c3, 'yyyy-MM-dd') AS TIMESTAMP)) created_at",
    "regexp_replace(_c4, '\r', '') status",
)

In [ ]:
applicationStages.write.parquet("/user/juliotorres/applicationStages", mode=mode)

```python
applicationStages.write.jdbc(url=url, table="applicationStages", mode=mode, properties=properties)
```

In [ ]:
hiveContext.registerDataFrameAsTable(stages, 'stages')
hiveContext.registerDataFrameAsTable(vacants, 'vacants')
hiveContext.registerDataFrameAsTable(candidates, 'candidates')
hiveContext.registerDataFrameAsTable(applications, 'applications')
hiveContext.registerDataFrameAsTable(applicationStages, 'applicationStages')

In [ ]:
history = hiveContext.sql("""
    select * 
      from candidates b
 left join applications a on a.candidate_id = b.id
 left join vacants c on a.vacant_id = c.id
 left join applicationStages d on d.application_id = a.id
 left join stages e on d.stage_id = e.id
""")

In [ ]:
candidates_c = candidates.columns
candidates_c[:3]

In [ ]:
applications_c = [ 'application_' + x for x in applications.columns ]
applications_c[:3]

In [ ]:
vacants_c = [ 'vacant_' + x for x in vacants.columns ]
vacants_c[:3]

In [ ]:
applicationStages_c = [ 'applicationStage_' + x for x in applicationStages.columns ]
applicationStages_c[:3]

In [ ]:
stages_c = [ 'stage_' + x for x in stages.columns ]
stages_c[:3]

In [ ]:
fields = candidates_c + applications_c + vacants_c + applicationStages_c + stages_c

In [ ]:
history = history.toDF(*fields)

In [ ]:
history.limit(10).toPandas().head()

In [ ]:
history.write.parquet("/user/juliotorres/history", mode=mode)

In [ ]:
hiveContext.sql("show tables").show()